<a href="https://colab.research.google.com/github/boxed-mikann/transformer_brain_encoder/blob/main/my_colab_file/algonauts_2023_challenge_tutorial_mine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
platform = 'colab' #@param ['colab', 'jupyter_notebook'] {allow-input: true}

In [2]:
if platform == 'colab':
    !pip install matplotlib==3.5.2
    !pip install nilearn==0.9.2

In [3]:
import os
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import matplotlib
matplotlib.use('Agg') # Force a non-interactive backend to potentially resolve conflicts
from matplotlib import pyplot as plt
from nilearn import datasets
from nilearn import plotting
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr as corr

In [4]:
if platform == 'colab':
    from google.colab import drive
    drive.mount('/content/drive/', force_remount=True)
    data_dir = '/content/drive/MyDrive/algonauts_2023_tutorial_data' #@param {type:"string"}
    parent_submission_dir = '/content/drive/MyDrive/algonauts_2023_challenge_submission' #@param {type:"string"}
    result_dir = '/content/drive/MyDrive/algonauts_2023/transformer_brain_encoder_results' #@param {type:"string"}

Mounted at /content/drive/


In [5]:
device = 'cpu' #@param ['cpu', 'cuda'] {allow-input: true}
device = torch.device(device)

In [6]:
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00


In [7]:
REPO_DIR = '/content/transformer_brain_encoder'
if not os.path.exists(REPO_DIR):
    !cd /content && git clone https://github.com/Hosseinadeli/transformer_brain_encoder.git
#!python main.py --run 1  --subj 1 --enc_output_layer 1 --readout_res 'rois_all'

Cloning into 'transformer_brain_encoder'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 246 (delta 6), reused 10 (delta 4), pack-reused 231 (from 1)
Receiving objects: 100% (246/246), 51.72 MiB | 23.58 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [16]:
class argObj:
  def __init__(self, data_dir, parent_submission_dir, subj):

    self.subj = format(subj, '02')
    self.data_dir = os.path.join(data_dir, 'subj'+self.subj)
    self.parent_submission_dir = parent_submission_dir
    self.subject_submission_dir = os.path.join(self.parent_submission_dir,
        'subj'+self.subj)

    # Create the submission directory if not existing
    if not os.path.isdir(self.subject_submission_dir):
        os.makedirs(self.subject_submission_dir)

args = argObj(data_dir, result_dir, 1)

In [20]:
args.data_dir

'/content/drive/MyDrive/algonauts_2023_tutorial_data/subj01'

In [19]:
!cd /content/transformer_brain_encoder && python main.py \
  --run 1  \
  --subj $args.subj \
  --enc_output_layer 1 \
  --readout_res 'rois_all' \
  --data_dir $data_dir \
  --output_path $args.subject_submission_dir

cpu
roi_name_maps: [{0: 'Unknown', 1: 'V1v', 2: 'V1d', 3: 'V2v', 4: 'V2d', 5: 'V3v', 6: 'V3d', 7: 'hV4'}, {0: 'Unknown', 1: 'EBA', 2: 'FBA-1', 3: 'FBA-2', 4: 'mTL-bodies'}, {0: 'Unknown', 1: 'OFA', 2: 'FFA-1', 3: 'FFA-2', 4: 'mTL-faces', 5: 'aTL-faces'}, {0: 'Unknown', 1: 'OPA', 2: 'PPA', 3: 'RSC'}, {0: 'Unknown', 1: 'OWFA', 2: 'VWFA-1', 3: 'VWFA-2', 4: 'mfs-words', 5: 'mTL-words'}, {0: 'Unknown', 1: 'early', 2: 'midventral', 3: 'midlateral', 4: 'midparietal', 5: 'ventral', 6: 'lateral', 7: 'parietal'}]
lh_challenge_rois: 6
lh_challenge_rois_s: torch.Size([25, 19004])
Training stimulus images: 8857
Validation stimulus images: 984

Test stimulus images: 159
Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100% 330M/330M [00:01<00:00, 316MB/s]
Traceback (most recent call last

In [9]:
#!cd /content/transformer_brain_encoder && python main.py \
  --subj args.subj \
  --run 1 \
  --data_dir '/content/drive/MyDrive/algonauts_2023_tutorial_data' \
  --dataset 'nsd_algo' \
  --backbone_arch 'dinov2_q' \
  --encoder_arch 'spatial_feature' \
  --readout_res 'rois_all' \
  --epochs 2 \
  --batch_size 8 \
  --image_size 224 \
  --output_path './results/' \
  --save_model 0 \
  --enc_output_layer 1 \
  --lr 0.0005 \
  --lr_drop 10

cpu
roi_name_maps: [{0: 'Unknown', 1: 'V1v', 2: 'V1d', 3: 'V2v', 4: 'V2d', 5: 'V3v', 6: 'V3d', 7: 'hV4'}, {0: 'Unknown', 1: 'EBA', 2: 'FBA-1', 3: 'FBA-2', 4: 'mTL-bodies'}, {0: 'Unknown', 1: 'OFA', 2: 'FFA-1', 3: 'FFA-2', 4: 'mTL-faces', 5: 'aTL-faces'}, {0: 'Unknown', 1: 'OPA', 2: 'PPA', 3: 'RSC'}, {0: 'Unknown', 1: 'OWFA', 2: 'VWFA-1', 3: 'VWFA-2', 4: 'mfs-words', 5: 'mTL-words'}, {0: 'Unknown', 1: 'early', 2: 'midventral', 3: 'midlateral', 4: 'midparietal', 5: 'ventral', 6: 'lateral', 7: 'parietal'}]
lh_challenge_rois: 6
lh_challenge_rois_s: torch.Size([25, 19004])
Traceback (most recent call last):
  File "/content/transformer_brain_encoder/main.py", line 491, in <module>
    main(0, 1, args)
  File "/content/transformer_brain_encoder/main.py", line 258, in main
    train_loader, val_loader = fetch_dataloaders(args, train='train')
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/transformer_brain_encoder/datasets/nsd.py", line 251, in fetch_dat